In [0]:
%sh
pip install spacy
pip install habanero
python -m spacy download en_core_web_sm

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-dcdd9388-043c-4063-bf11-aa4ad0f4e40e
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dcdd9388-043c-4063-bf11-aa4ad0f4e40e/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dcdd9388-043c-4063-bf11-aa4ad0f4e40e/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dcdd9388-043c-4063-bf11-aa4ad0f4e40e/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
dbutils.library.restartPython()

In [0]:
from pyspark.sql.types import StringType, IntegerType, ArrayType
import time
import pyspark.sql.functions as F
from pyspark.sql import Window
import requests
import json
import urllib.parse
import spacy

#### Database Schema

<img src="https://i.ibb.co/hRqLR8d/Schema.png" alt="Schema" border="0">

In [0]:
raw_df = spark.read.format("delta").load("/user/hive/warehouse/scientific_publications")
display(raw_df.limit(5))

_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year
53e99784b7602d9701f3e3f5,null,null,null,null,null,null,null,List(),en,0,null,null,null,null,3GIO.,null,"List(null, null, null, null, null, null, null, null, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e133,"Drought is the first place in all the natural disasters in the world. It is especially serious in North China Plain. In this paper, different soil water content control levels at winter wheat growth stages are performed on Gucheng Ecological-Meteorological Integrated Observation Experiment Station of CAMS, China. Some canopy parameters, including growth conditions, dry weight, physiological parameters and hyperspectral reflectance, are measured from erecting stage to milk stage for winter wheat in 2009. The relationship between canopy parameters and soil relative moisture, canopy water content and water indices of winter wheat are established. The results show that some parameters, such as SPAD and dry weight of leaves, decrease with the increasing of soil relative moisture, while other parameters, including dry weight of caudexes, above ground dry weight, height, photosynthesis rate, intercellular CO 2 concentration, stomatal conductance and transpiration rate, increase corresponding to the soil relative moisture. Obvious linear relationship between stomatal conductance and transpiration rate is established with 45 samples, which R2 reaches to 0.6152. Finally, the fitting equations between canopy water content and water indices are regressed with b5, b6 and b7 of MODIS bands. The equations are best with b7 and worst with b5. So the fitting equations with b7 can be used to inverse the canopy water content of winter wheat using MODIS or other remote sensing images with similar bands range to MODIS in Hebei Province. © 2011 IEEE.","List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",,,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),"List(53a7297d20f7420be8bd4ae7, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e151,,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",,,,"List(handwriting recognition, prototypes, image segmentation, com

In [0]:
#Filter out titles that have less than 2 words
filtered_df = raw_df.withColumn("title_word_count", F.size(F.split("title", " "))).filter("title_word_count > 1")
# filtered_df = filtered_df.filter(F.col("doi").isNull())
# filtered_df = filtered_df.filter("n_citation > 3")
display(filtered_df.limit(200))

_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year,title_word_count
53e99967b7602d97021a560a,"Ab initio predictions of secondary structures in proteins have to combine local predictions, based on short fragments of the protein sequence, with consistency restrictions, as not all locally plausible predictions may be simultaneously true. We use the fact that secondary structures are patterns of hydrogen bonds and that a single residue can participate in hydrogen bonds of at most one secondary structure. Consistency of fixed-sized pieces of secondary structures is the easiest to approximate and we formalize it as 1-2 matching problem. Consistency of entire secondary structures is a version of set packing. We also investigate how to form a simple problem if we add the requirement that the secondary structure and the loops that connect them fit together in a metric space. Every problem that we investigated is MAX-SNP hard and it has a constant factor approximation. Computational experience suggests that in biological instances, we can find nearly optimal solutions using heuristics.","List(List(53f45d3adabfaee0d9c0cb90, null, berman@cse.psu.edu, 5b86b6c2e1cd8e14a34ca959, Piotr Berman, null, null, null, null, The Pennsylvania State University, University Park, PA, USA, null, 5f71b2d91c455f439fe3eb4f, List(Penn State Univ, University Pk, PA 16802 USA), null, null), List(53f436cbdabfaedf43582e9f, null, null, 5b86b6c2e1cd8e14a34ca959, Jieun Jeong, null, null, null, null, The Pennsylvania State University, University Park, PA, USA, null, 5f71b2d91c455f439fe3eb4f, List(Penn State Univ, University Pk, PA 16802 USA), null, null))",10.1007/s00453-007-9068-8,"List(Algorithm, Heuristics, Set packing, Metric space, Ab initio, Hydrogen bond, Protein secondary structure, Mathematics, Protein structure, Theory of computation)",,0178-4617,1,"List(Secondary Structure, Maximum Weight, Local Prediction, Horizontal Edge, Connector Edge)",en,2,34,16,null,"List(53e9bdcdb7602d9704a7e6ae, 53e99d5eb7602d970261a1c4, 53e9bdcdb7602d9704a7e6ae, 53e99e3eb7602d970270422e, 53e9adffb7602d970380a762, 53e9b593b7602d97040d9071, 53e99c6fb7602d9702522122, 55a43e58612ca648688dc943, 53e99822b7602d97020431be)",Consistent Sets of Secondary Structures in Proteins,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)",53,2009,7
53e99960b7602d97021a53fa,"In this work a new method of feature extraction for an interactive and adaptive recognizer for on-line handwritten alphanumeric characters has been proposed. The system is suitable for use in conjunction with magnetic pen based devices for inputting data to a data processing system or a computer terminal. The features are extracted from dynamically changing locations of the writing device. The new method of feature extraction is simple, computationally light and fast enough for adaptive on-line use. Extracted features are robust with respect to all possible distortions like shape, size, and orientation. For simulation experiment, numerals 0-9 are used. A single hidden layer feed forward neural network trained by Quickprop algorithm, a variation of error back propagation is used for recognition. Very high recognition rates, even for highly distorted samples have been achieved confirming high generalization capability of the extracted feature set.","List(List(53f36626dabfae4b3499aa7a, null, null, 5b86b9c0e1cd8e14a3625bbe, Basabi Chakraborty, null, null, null, null, Faculty of Software and Information Science, Iwate Prefectural University, 152-52 Aza Sugo, Takizawamura, Iwate 020-0193, Japan, null, 5f71b4211c455f439fe47d1e,

## Authors

In [0]:
def get_author_from_dblp(title, rank):
    URL = "http://dblp.org/search/publ/api?q=" + title.replace(" ", "+") + "&format=json"
    try:
        r = requests.get(url = URL)
        data = r.json()
        return data['result']['hits']['hit'][0]['info']['authors']['author'][rank-1]['text']
    except:
        return None

In [0]:
authors = (filtered_df.limit(100).select("doi", "authors", "title", F.posexplode(F.col("authors")).alias("rank", "authors_exp"))
            .withColumn("rank", F.col("rank") + 1)
            .select("authors_exp.*","*")
            .select("doi", "rank", "name", "title")
            .withColumn("name", F.initcap(F.col("name")))
            )

authors2 = authors.filter(F.col("name").rlike(r"^\p{L}\.?(-\p{L}\.?)?\s.+"))
authors_rdd = authors2.rdd.map(lambda x: (x[0], x[1], x[2], x[3], get_author_from_dblp(x[3], x[1])))
authors2 = authors_rdd.toDF(["doi", "rank", "name", "title", "dblp_name"])
    
authors2 = (authors2.withColumn("dblp_name", F.when(F.col("dblp_name") == "null", F.col("name")).otherwise(F.col("dblp_name")))
           .withColumn("dblp_name", F.regexp_extract(F.col("dblp_name"), r"^(\D+)(\s\d\d\d\d)?$", 1)))

authors_raw = (authors.withColumn("dblp_name", F.col("name"))
           .union(authors2)
           .withColumn("last_name_raw", F.regexp_extract(F.col("name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("dblp_name", F.when(F.col("dblp_name") == "null", F.col("name")).otherwise(F.col("dblp_name")))
           .withColumn("dblp_name", F.regexp_extract(F.col("dblp_name"), r"^(\D+)(\s\d\d\d\d)?$", 1))
           .withColumn("dblp_last_name", F.regexp_extract(F.col("dblp_name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("name", F.when(F.col("last_name_raw") == F.col("dblp_last_name"), 
                                      F.col("dblp_name")).otherwise(F.col("name")))
           .withColumn("first_name", F.regexp_extract(F.col("name"), r"^(\S+)\s.+$", 1))
           .withColumn("last_name", F.regexp_extract(F.col("name"), r"^.+\s(\S+)$", 1))
           .withColumn("middle_name", F.regexp_extract(F.col("name"), r"^\S+\s(\S+)\s\S+$", 1))
          )

authors = (authors_raw.select("first_name", "last_name", "middle_name")
            .dropDuplicates()
            .withColumn("author_id", F.monotonically_increasing_id())
          )

authors = (authors_raw.join(authors, ["first_name", "last_name", "middle_name"])
           .select("author_id", "first_name", "last_name", "middle_name", "title", F.col("name").alias("author"), "rank")
           .dropDuplicates()
          )

display(authors)

authors_clean = authors.select("author_id", "first_name", "last_name", "middle_name")
authors_clean.write.format("delta").mode("overwrite").saveAsTable("authors")

author_id,first_name,last_name,middle_name,title,author,rank
0,Andreas,Herzig,,Dynamic Logic of Propositional Assignments: A Well-Behaved Variant of PDL,Andreas Herzig,2
1,Fu-lai,Chung,,Stock time series pattern matching: Template-based vs. rule-based approaches,Fu-lai Chung,2
2,Ravi,Sundaram,,Secure and scalable match: overcoming the universal circuit bottleneck using group programs.,Ravi Sundaram,2
3,Qingtang,Liu,,Research on DRM-enabled learning objects model,Qingtang Liu,1
4,Baruch,Mazor,,A trellis code construction and coding theorem for stationary Gaussian sources,Baruch Mazor,1
5,Han-chin,Cheng,,Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.,Han-chin Cheng,5
6,Weiwu,Shi,,Communication with Threads in Software DSMs,Weiwu Hu Gang Shi,1
7,Peng,Liu,,In-broker access control: towards efficient end-to-end performance of information brokerage systems,Peng Liu,3
8,I.,Prelog,,A distributed virtual reality-based system for neonatal decision-making training,I. Prelog,3
9,Daniel,Pellicer,,Combinatorial Structure of Schulte's Chiral Polyhedra,Daniel Pellicer,1


## Organizations

In [0]:
def get_organization(name, country):
    try:
        URL = "https://api.ror.org/organizations?query=" + name.replace(" ", "+")
        r = requests.get(url = URL)
        data = r.json()
        if data['number_of_results'] == 0:
            return "No results"
        for i in range(data['number_of_results']):
            found_country = data['items'][i]["country"]["country_name"]
            if country == "" or country == found_country:
                return data['items'][i]["name"]+";"+data['items'][i]["addresses"][0]["city"]+";"+found_country
    except:
        return "No results"

In [0]:
countries = "(,|\s)(Afghanistan|Albania|Algeria|Andorra|Angola|Antigua and Barbuda|Argentina|Armenia|Australia|Austria|Azerbaijan|Bahamas|Bahrain|Bangladesh|Barbados|Belarus|Belgium|Belize|Benin|Bhutan|Bolivia|Bosnia and Herzegovina|Botswana|Brazil|Brunei|Bulgaria|Burkina Faso|Burundi|Cabo Verde|Cambodia|Cameroon|Canada|Central African Republic|Chad|Chile|China|Colombia|Comoros|Democratic Republic of the Congo|Republic of the Congo|Costa Rica|Cote d'Ivoire|Croatia|Cuba|Cyprus|Czech Republic|Denmark|Djibouti|Dominica|Dominican Republic|Ecuador|Egypt|El Salvador|Equatorial Guinea|Eritrea|Estonia|Ethiopia|Fiji|Finland|France|Gabon|Gambia|Georgia|Germany|Ghana|Greece|Grenada|Guatemala|Guinea|Guinea-Bissau|Guyana|Haiti|Honduras|Hungary|Iceland|India|Indonesia|Iran|Iraq|Ireland|Israel|Italy|Jamaica|Japan|Jordan|Kazakhstan|Kenya|Kiribati|Kosovo|Kuwait|Kyrgyzstan|Laos|Latvia|Lebanon|Lesotho|Liberia|Libya|Liechtenstein|Lithuania|Luxembourg|North Macedonia|Madagascar|Malawi|Malaysia|Maldives|Mali|Malta|Marshall Islands|Mauritania|Mauritius|Mexico|Micronesia|Moldova|Monaco|Mongolia|Montenegro|Morocco|Mozambique|Myanmar|Namibia|Nauru|Nepal|Netherlands|New Zealand|Nicaragua|Niger|Nigeria|North Korea|Norway|Oman|Pakistan|Palau|Palestine|Panama|Papua New Guinea|Paraguay|Peru|Philippines|Poland|Portugal|Qatar|Romania|Russia|Rwanda|Saint Kitts and Nevis|Saint Lucia|Saint Vincent and the Grenadines|Samoa|San Marino|Sao Tome and Principe|Saudi Arabia|Senegal|Serbia|Seychelles|Sierra Leone|Singapore|Slovakia|Slovenia|Solomon Islands|Somalia|South Africa|South Korea|South Sudan|Spain|Sri Lanka|Sudan|Suriname|Swaziland|Sweden|Switzerland|Syria|Taiwan|Tajikistan|Tanzania|Thailand|Timor-Leste|Togo|Tonga|Trinidad and Tobago|Tunisia|Turkey|Turkmenistan|Tuvalu|Uganda|Ukraine|United Arab Emirates|UAE|United Kingdom|UK|United States of America|USA|United States|US|Uruguay|Uzbekistan|Vanuatu|Vatican City|Venezuela|Vietnam|Yemen|Zambia|Zimbabwe)(,|\s|$)"


organization = (filtered_df.limit(100).select("authors", "title", F.explode(F.col("authors")).alias("authors_exp"))
               .select("authors_exp.*","*")
               .withColumn("author", F.col("name"))
               .select("author", "org")
               .filter(F.col("org").isNotNull())
               .dropDuplicates()
                # ., +, *, ?, ^, $, (, ), [, ], {, }, |, \
               .withColumn("strip_org", F.regexp_replace(F.col("org"), r'[\+-=#&\|><!\(\)\{\}\[\]\^"~\*\?:\\/]', " "))
               .withColumn("county", F.regexp_extract(F.col("org"), countries, 2))
               .withColumn("county", F.regexp_replace("county", "United States of America|USA|US", "United States"))
               .withColumn("county", F.regexp_replace("county", "UK", "United Kingdom"))
               .withColumn("county", F.regexp_replace("county", "UAE", "United Arab Emirates"))
               )

organization_rdd = organization.rdd.map(lambda x: (x[0], x[1], x[2], x[3], get_organization(x[2], x[3])))
organization_raw = (organization_rdd.toDF(["author", "org", "strip_org", "country", "api_org"])
                    .withColumn("api_name", F.regexp_extract(F.col("api_org"), r"^(.+);.+;.+$", 1))
                    .withColumn("match", F.expr(r"regexp_extract(org, concat('(', api_name,')'), 0)"))
                    .withColumn("name", F.when(F.col("match") == "", F.col("org"))
                               .otherwise(F.col("api_name")))
                    .withColumn("city", F.when(F.col("match") == "", "")
                               .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;(.+);.+$", 1)))
                    .withColumn("country", F.when(F.col("match") == "", F.col("country"))
                               .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;.+;(.+)$", 1)))
                   )

organization = (organization_raw.select("name", "city", "country")
                .dropDuplicates()
                .withColumn("org_id", F.monotonically_increasing_id())
               )

organization = (organization_raw.join(organization, ["name", "city", "country"])
                .select("org_id", "name", "city", "country", "author"))

display(organization)

organization_clean = organization.select("org_id", "name", "city", "country")
organization_clean.write.format("delta").mode("overwrite").saveAsTable("organization")

org_id,name,city,country,author
0,"Tokyo Inst. Tech., Yokohama, Japan",,Japan,Takao Terano
1,Georgia Institute of Technology,Atlanta,United States,Farshid Delgosha
1,Georgia Institute of Technology,Atlanta,United States,Erman Ayday
1,Georgia Institute of Technology,Atlanta,United States,Faramarz Fekri
2,"Universit ¨ , at Karlsruhe",,,Gerhard Starke
3,National University of Singapore,Singapore,Singapore,Yong Zeng
3,National University of Singapore,Singapore,Singapore,Zhifeng Bao
3,National University of Singapore,Singapore,Singapore,Tok Wang Ling
3,National University of Singapore,Singapore,Singapore,Luochen Li
4,York University Toronto Ontario M3J 1P3 Canada,,Canada,Daniel Pellicer


## Publications

In [0]:
publication_raw = filtered_df.limit(100).select("_id", "title", "volume", "n_citation", "doi", "url")

display(publication_raw)

_id,title,volume,n_citation,doi,url
53e99967b7602d97021a560a,Consistent Sets of Secondary Structures in Proteins,53,2,10.1007/s00453-007-9068-8,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)"
53e99960b7602d97021a53fa,A new feature extraction technique for on-line recognition of handwritten alphanumeric characters,148,19,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)"
53e99960b7602d97021a5416,King John Divided,19,0,10.1093/llc/19.2.181,List(http://dx.doi.org/10.1093/llc/19.2.181)
53e99960b7602d97021a541f,Peer Assist Live Streaming Overlay for Next-Generation-Networks,1,0,10.4018/jhcr.2010100102,"List(http://dx.doi.org/10.4018/jhcr.2010100102, http://dx.doi.org/10.4018/978-1-4666-0921-1.ch017)"
53e99960b7602d97021a5427,Paramecium: Assembling Raw Nodes into Composite Cells,3222,2,10.1007/978-3-540-30141-7_67,"List(http://dx.doi.org/10.1007/978-3-540-30141-7_67, http://www.webofknowledge.com/)"
53e99967b7602d97021a5645,Radar Signal Retrodiffusion by Water Surface,2,1,10.1109/IGARSS.2009.5418122,"List(http://dx.doi.org/10.1109/IGARSS.2009.5418122, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=5418122)"
53e99960b7602d97021a543e,Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.,,5,10.1109/ICSMC.2008.4811630,"List(http://dx.doi.org/10.1109/ICSMC.2008.4811630, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4811630)"
53e99967b7602d97021a566b,Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation,,13,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314)
53e99960b7602d97021a5451,Weighted census transform for feature representation,,2,10.1109/URAI.2013.6677409,"List(http://dx.doi.org/10.1109/URAI.2013.6677409, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=6677409)"
53e99967b7602d97021a5814,A signaling and control architecture for mobility support in wireless ATM networks,1,134,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)"


In [0]:
def check_return_data(check_type, data, cur):
    if data is not None and check_type in data:
        return data[check_type]
    else:
        return cur
    
def update_df(df, doi_list, data_list, data_str, is_int=False):
    data_dict = dict(zip(doi_list, data_list))
    update_data = udf(lambda x: data_dict[x], IntegerType() if is_int else StringType())
    return df.withColumn(data_str, update_data(F.col('doi')))

def get_publication_data(df):
    doi_list = df.select(F.col("doi")).rdd.flatMap(lambda x: x).collect()
    url_list = df.select(F.col("url")).rdd.flatMap(lambda x: x).collect()
    title_list = df.select(F.col("title")).rdd.flatMap(lambda x: x).collect()
    citation_list = df.select(F.col("n_citation")).rdd.flatMap(lambda x: x).collect()
    volume_list = df.select(F.col("volume")).rdd.flatMap(lambda x: x).collect()
    new_volume_list = []
    n_citation_list = []
    series_list = []
    new_doi_list = []
    for i, doi in enumerate(doi_list):
        try:
            if doi == "" or doi is None:
                if "doi" in url_list[i][0]:
                    doi_req = url_list[i][0].split("org/")[-1]
                else:
                    raise Exception
            else:
                doi_req = doi
            
            new_doi_list.append(doi_req)
            response = requests.get(f"https://api.crossref.org/works/{doi_req}")
            data = response.json()['message']
        except Exception:
            new_doi_list.append(None)
            data = None

        n_citation_list.append(int(check_return_data('is-referenced-by-count', data, citation_list[i])))
        new_volume_list.append(check_return_data('volume', data, volume_list[i]))
        temp = check_return_data('container-title', data, None)
        series_list.append(temp if temp == None else temp[0])

        time.sleep(0.05)
    
    df = update_df(df, doi_list, n_citation_list, 'n_citation')
    df = update_df(df, doi_list, new_volume_list, 'volume')
    df = update_df(df, doi_list, series_list, 'series')
    df = update_df(df, doi_list, new_doi_list, 'doi')
    
    return df

In [0]:
publications_raw = get_publication_data(publication_raw).filter("n_citation > 2")
# publication = get_publication_dataget_crossref_data(publication_raw).select("_id", "title", "volume", "series", "n_citation").filter("n_citation > 2")

publications = (publications_raw.select("title", "volume", "series", "n_citation")
            .dropDuplicates()
            .withColumn("publ_id", F.monotonically_increasing_id())
          )

publications = (publications_raw.join(publications, ["title", "volume", "series", "n_citation"])
           .select("publ_id", "title", "volume", "series", "n_citation")
           .dropDuplicates()
          )

display(publications)
publications.write.format("delta").mode("overwrite").saveAsTable("publications")

publ_id,title,volume,series,n_citation
3,Error rate reduction in DNA self-assembly by non-constant monomer concentrations and profiling,,"2007 Design, Automation & Test in Europe Conference & Exhibition",3
15,Stock time series pattern matching: Template-based vs. rule-based approaches,20,Engineering Applications of Artificial Intelligence,84
54,Replacing square roots by Pythagorean sums,27,IBM Journal of Research and Development,9
57,Fast and scalable enrollment for face identification based on Partial Least Squares.,,2013 10th IEEE International Conference and Workshops on Automatic Face and Gesture Recognition (FG),5
37,A hybrid fem-based method for aligning prone and supine images for image guided breast surgery,,2011 IEEE International Symposium on Biomedical Imaging: From Nano to Macro,14
56,The complete proof on the economic order quantity under conditions of permissible delay in payments,59,Journal of the Operational Research Society,3
38,Agglomerative hierarchical clustering with constraints: theoretical and empirical results,3721,Knowledge Discovery in Databases: PKDD 2005,109
32,Reducing mass degeneracy in SAR by MS by stable isotopic labeling.,8,Journal of Computational Biology,9
12,Design of soft robotic actuators using fluid-filled fiber-reinforced elastomeric enclosures in parallel combinations.,,2012 IEEE/RSJ International Conference on Intelligent Robots and Systems,77
42,Improving Learner Performance with Data Sampling and Boosting,1,2008 20th IEEE International Conference on Tools with Artificial Intelligence,3


## Types

In [0]:
types_raw = filtered_df.limit(100).select("_id", "volume", "issue", "doi", "url", "venue", "title")

display(types_raw)

_id,volume,issue,doi,url,venue,title
53e99967b7602d97021a560a,53,1,10.1007/s00453-007-9068-8,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)",Consistent Sets of Secondary Structures in Proteins
53e99960b7602d97021a53fa,148,1-4,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)","List(555036b77cea80f95414b7e3, null, null, null, null, null, null, Inf. Sci., null, null, null, null, 0)",A new feature extraction technique for on-line recognition of handwritten alphanumeric characters
53e99960b7602d97021a5416,19,2,10.1093/llc/19.2.181,List(http://dx.doi.org/10.1093/llc/19.2.181),"List(555036d37cea80f95415b0b3, null, null, null, null, null, null, LLC, null, null, null, null, 0)",King John Divided
53e99960b7602d97021a541f,1,4,10.4018/jhcr.2010100102,"List(http://dx.doi.org/10.4018/jhcr.2010100102, http://dx.doi.org/10.4018/978-1-4666-0921-1.ch017)","List(555036cc7cea80f954158149, null, null, International Journal of Handheld Computing Research, null, null, null, IJHCR, null, null, null, null, 0)",Peer Assist Live Streaming Overlay for Next-Generation-Networks
53e99960b7602d97021a5427,3222,,10.1007/978-3-540-30141-7_67,"List(http://dx.doi.org/10.1007/978-3-540-30141-7_67, http://www.webofknowledge.com/)","List(null, 0302-9743, null, null, null, 1611-3349, null, LECTURE NOTES IN COMPUTER SCIENCE, null, LECTURE NOTES IN COMPUTER SCIENCE, null, J, null)",Paramecium: Assembling Raw Nodes into Composite Cells
53e99967b7602d97021a5645,2,,10.1109/IGARSS.2009.5418122,"List(http://dx.doi.org/10.1109/IGARSS.2009.5418122, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=5418122)","List(53a7297c20f7420be8bd4ae5, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",Radar Signal Retrodiffusion by Water Surface
53e99960b7602d97021a543e,,,10.1109/ICSMC.2008.4811630,"List(http://dx.doi.org/10.1109/ICSMC.2008.4811630, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4811630)","List(555037277cea80f954176d3b, null, null, Systems, Man and Cybernetics, null, null, null, SMC, null, null, null, null, 0)",Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.
53e99967b7602d97021a566b,,,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314),"List(53a726bb20f7420be8b7f846, null, null, International Conference on Robotics and Automation, null, null, null, ICRA, null, null, null, null, 0)",Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation
53e99960b7602d97021a5451,,,10.1109/URAI.2013.6677409,"List(http://dx.doi.org/10.1109/URAI.2013.6677409, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=6677409)","List(555037017cea80f95416d334, null, null, International Conference on Ubiquitous Robots and Ambient Intelligence, null, null, null, URAI, null, null, null, null, 0)",Weighted census transform for feature representation
53e99967b7602d97021a5814,1,3,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)","List(53a730a020f7420be8cff2d8, null, null, International Conference on Communications, null, null, null, MONET, null, null, null, null, 0)",A signaling and control architecture for mobility support in wireless ATM networks


In [0]:
types_pre_api = types_raw.withColumn("type", F.when(F.col("venue.raw").contains("@"), "workshop")
                   .when(((F.col("volume").isNotNull()) & (F.col("volume") != "")) | ((F.col("issue").isNotNull()) & (F.col("issue") != "")), "journal-article")
                   .otherwise("conference paper"))

display(types_pre_api)

_id,volume,issue,doi,url,venue,title,type
53e99967b7602d97021a560a,53,1,10.1007/s00453-007-9068-8,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)",Consistent Sets of Secondary Structures in Proteins,journal-article
53e99960b7602d97021a53fa,148,1-4,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)","List(555036b77cea80f95414b7e3, null, null, null, null, null, null, Inf. Sci., null, null, null, null, 0)",A new feature extraction technique for on-line recognition of handwritten alphanumeric characters,journal-article
53e99960b7602d97021a5416,19,2,10.1093/llc/19.2.181,List(http://dx.doi.org/10.1093/llc/19.2.181),"List(555036d37cea80f95415b0b3, null, null, null, null, null, null, LLC, null, null, null, null, 0)",King John Divided,journal-article
53e99960b7602d97021a541f,1,4,10.4018/jhcr.2010100102,"List(http://dx.doi.org/10.4018/jhcr.2010100102, http://dx.doi.org/10.4018/978-1-4666-0921-1.ch017)","List(555036cc7cea80f954158149, null, null, International Journal of Handheld Computing Research, null, null, null, IJHCR, null, null, null, null, 0)",Peer Assist Live Streaming Overlay for Next-Generation-Networks,journal-article
53e99960b7602d97021a5427,3222,,10.1007/978-3-540-30141-7_67,"List(http://dx.doi.org/10.1007/978-3-540-30141-7_67, http://www.webofknowledge.com/)","List(null, 0302-9743, null, null, null, 1611-3349, null, LECTURE NOTES IN COMPUTER SCIENCE, null, LECTURE NOTES IN COMPUTER SCIENCE, null, J, null)",Paramecium: Assembling Raw Nodes into Composite Cells,journal-article
53e99967b7602d97021a5645,2,,10.1109/IGARSS.2009.5418122,"List(http://dx.doi.org/10.1109/IGARSS.2009.5418122, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=5418122)","List(53a7297c20f7420be8bd4ae5, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",Radar Signal Retrodiffusion by Water Surface,journal-article
53e99960b7602d97021a543e,,,10.1109/ICSMC.2008.4811630,"List(http://dx.doi.org/10.1109/ICSMC.2008.4811630, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4811630)","List(555037277cea80f954176d3b, null, null, Systems, Man and Cybernetics, null, null, null, SMC, null, null, null, null, 0)",Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.,conference paper
53e99967b7602d97021a566b,,,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314),"List(53a726bb20f7420be8b7f846, null, null, International Conference on Robotics and Automation, null, null, null, ICRA, null, null, null, null, 0)",Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation,conference paper
53e99960b7602d97021a5451,,,10.1109/URAI.2013.6677409,"List(http://dx.doi.org/10.1109/URAI.2013.6677409, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=6677409)","List(555037017cea80f95416d334, null, null, International Conference on Ubiquitous Robots and Ambient Intelligence, null, null, null, URAI, null, null, null, null, 0)",Weighted census transform for feature representation,conference paper
53e99967b7602d97021a5814,1,3,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)","List(53a730a020f7420be8cff2d8, null, null, International Conference on Communications, null, null, null, MONET, null, null, null, null, 0)",A signaling and control architecture for mobility support in w

In [0]:
def check_return_data(check_type, data, cur):
    if data is not None and check_type in data:
        return data[check_type]
    else:
        return cur
    
def update_df(df, doi_list, data_list, data_str, is_int=False):
    new_df = spark.createDataFrame(zip(df.select("doi").rdd.flatMap(lambda x: x).collect(), data_list), ["doi", data_str])
    return df.join(new_df, on="doi", how="left").select(new_df["doi"], new_df[data_str], F.col("title"))

def get_type_data(df):
    doi_list = df.select(F.col("doi")).rdd.flatMap(lambda x: x).collect()
    url_list = df.select(F.col("url")).rdd.flatMap(lambda x: x).collect()
    type_list = df.select(F.col("type")).rdd.flatMap(lambda x: x).collect()
    new_type_list = []
    for i, doi in enumerate(doi_list):
        try:
            if doi == "" or doi is None:
                if "doi" in url_list[i][0]:
                    doi_req = url_list[i][0].split("org/")[-1]
                else:
                    raise Exception
            else:
                doi_req = doi
            
            response = requests.get(f"https://api.crossref.org/works/{doi_req}")
            data = response.json()['message']
        except Exception:
            data = None

        new_type_list.append(check_return_data('type', data, type_list[i]))

        time.sleep(0.05)

    df = update_df(df, doi_list, new_type_list, 'type')
    
    return df

In [0]:
types_raw = get_type_data(types_pre_api)

types = (types_raw.select("type")
            .dropDuplicates()
            .withColumn("type_id", F.monotonically_increasing_id())
          )

types = (types_raw.join(types, ["type"])
           .select("type_id", "type", "title")
           .dropDuplicates()
        )

display(types)

types_clean = types.select("type_id", "type")
types_clean.write.format("delta").mode("overwrite").saveAsTable("types")

type_id,type,title
0,journal-article,Consistent Sets of Secondary Structures in Proteins
0,journal-article,A new feature extraction technique for on-line recognition of handwritten alphanumeric characters
0,journal-article,King John Divided
0,journal-article,Peer Assist Live Streaming Overlay for Next-Generation-Networks
1,book-chapter,Paramecium: Assembling Raw Nodes into Composite Cells
2,proceedings-article,Radar Signal Retrodiffusion by Water Surface
2,proceedings-article,Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.
2,proceedings-article,Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation
2,proceedings-article,Weighted census transform for feature representation
0,journal-article,A signaling and control architecture for mobility support in wireless ATM networks


## Keywords

In [0]:
keywords_raw = filtered_df.limit(100).select("_id", "keywords", "doi", "title")

display(keywords_raw)

_id,keywords,doi,title
53e99967b7602d97021a560a,"List(Secondary Structure, Maximum Weight, Local Prediction, Horizontal Edge, Connector Edge)",10.1007/s00453-007-9068-8,Consistent Sets of Secondary Structures in Proteins
53e99960b7602d97021a53fa,"List(adaptive on-line use, data processing system, feature extraction, inputting data, on-line recognition, handwritten alphanumeric character, feature set, new feature extraction technique, extracted feature, adaptive recognizer, new method, high generalization capability, high recognition rate, feed forward neural network, error back propagation, simulation experiment, data processing, artificial neural network)",10.1016/S0020-0255(02)00276-1,A new feature extraction technique for on-line recognition of handwritten alphanumeric characters
53e99960b7602d97021a5416,List(),10.1093/llc/19.2.181,King John Divided
53e99960b7602d97021a541f,"List(scalability, ims, p2p, ngn, next generation network)",10.4018/jhcr.2010100102,Peer Assist Live Streaming Overlay for Next-Generation-Networks
53e99960b7602d97021a5427,List(),10.1007/978-3-540-30141-7_67,Paramecium: Assembling Raw Nodes into Composite Cells
53e99967b7602d97021a5645,"List(airborne radar, backscatter, height measurement, oceanographic regions, oceanographic techniques, remote sensing by radar, synthetic aperture radar, Camargue region, DRIVE radar, France, ONERA BUSARD platform, SAR instrument, backscattering coefficient profiles, dry surfaces, future space Ka band altimetry missions, nadir backscattering, pond surfaces, sea surfaces, sensor configuration, specular behavior, steep radar incident angle, synthetic aperture radar, water surface radar signal retrodiffusion, wet surfaces, Ka band, altimetry mode, radar sensor)",10.1109/IGARSS.2009.5418122,Radar Signal Retrodiffusion by Water Surface
53e99960b7602d97021a543e,"List(Java, Web services, biomedical telemetry, health care, mobile computing, sensor fusion, telemedicine, wireless sensor networks, Custos, ICT technique, Internet, Squawk Java J2ME, Sun SPOT, UPnP digital-home platform, Web service, Wei-Gong memorial hospital, clinical trial, geriatric psychiatry ward, mobile phone, multisensor data fusion, remote on-demand healthcare, small programmable object technology, smart phone, wireless sensor network, Smart Environments, Telemedicine, Telemonitoring, Wireless Sensor Networks)",10.1109/ICSMC.2008.4811630,Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.
53e99967b7602d97021a566b,"List(information technology, simulation, transportation, objective function, dynamic programming, state space, sensor network, elevators, uncertainty, testing, normal operator, look ahead, lagrangian relaxation, local search)",10.1109/ROBOT.2005.1570314,Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation
53e99960b7602d97021a5451,"List(weighted census transform, visual feature representation method, image representation, binary code bit, face recognition, ar face datasets, modified census transform, wct, statistical analysis, feature representation model, visual image classification, feature representation, training dataset, compressed sensing, image classification, statistical approach, mct, entropy information, transforms, entropy, compressive sensing technique, pattern classification)",10.1109/URAI.2013.6677409,Weighted census transform for feature representation
53e99967b7602d97021a5814,"List(Wireless Link, Mobile Terminal, Control Protocol, Control Architecture, Control Layer)",10.1007/BF01193262,A signaling and control architecture for mobility support in wireless ATM networks


In [0]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def is_city_location_person(keyword):
    # Process the keyword using the spaCy NER model
    doc = nlp(keyword)
    
    # Check if any of the entities recognized by the model are of type GPE (city or location) or PERSON
    for ent in doc.ents:
        if ent.label_ in ['GPE', 'PERSON']:
            return True
    
    return False

def update_df(df, doi_list, data_list, data_str, is_int=False):
    data_dict = dict(zip(doi_list, data_list))
    update_data = udf(lambda x: data_dict[x], IntegerType() if is_int else StringType())
    return df.withColumn(data_str, update_data(F.col('doi')))

def check_keywords(df):
    doi_list = df.select(F.col("doi")).rdd.flatMap(lambda x: x).collect()
    keywords_list = df.select(F.col("keywords")).rdd.flatMap(lambda x: x).collect()
    new_keywords_list = []
    for i, keys in enumerate(keywords_list):
        temp = []
        for k in keys:
            if not is_city_location_person(k):
                temp.append(k)
        new_keywords_list.append(temp)
        
    df = update_df(df, doi_list, keywords_list, 'keywords')
    
    return df

In [0]:
keywords_raw = check_keywords(keywords_raw).filter(F.size(keywords_raw.keywords) > 0)


keywords = (keywords_raw.select("keywords")
            .dropDuplicates()
            .withColumn("keyw_id", F.monotonically_increasing_id())
          )

keywords = (keywords_raw.join(keywords, ["keywords"])
           .select("keyw_id", "keywords", "title")
           .dropDuplicates()
        )

display(keywords)

keywords_clean = keywords.select("keyw_id", "keywords")
keywords_clean.write.format("delta").mode("overwrite").saveAsTable("keywords")

keyw_id,keywords,title
0,"[speaker model, multiple session, large data, large data set, incremental robust adaptation method, incremental adaptation method, inaccurate speaker model, robust speaker recognition, large amount, utterance variation, speaker recognition system, incremental verstion, speaker recognition, gaussian mixture model]",Robust speaker recognition against utterance variations
1,"[asymptotic behavior, constant coefficient, rational difference equation, phase portrait, rational type difference equation, attractor, phase space]",On the asymptotic behavior of a system of two rational difference equations
2,[distributed systems],A distributed virtual reality-based system for neonatal decision-making training
3,"[matrix decomposition, face, feature extraction, image classification, computational modeling, biometrics, face recognition, descriptive approach, vectors, eigenface]",Fast and scalable enrollment for face identification based on Partial Least Squares.
4,[bergman space],Schatten Class Toeplitz Operators on the Bergman Space
5,"[empirical result, feasibility problem, non-hierarchical clustering, agglomerative clustering, cluster-level constraint, traditional agglomerative algorithm, dead-end solution, hierarchical clustering, agglomerative hierarchical clustering, standard agglomerative algorithm, constraint combination, triangle inequality]",Agglomerative hierarchical clustering with constraints: theoretical and empirical results
6,"[orthogonal issue, in-broker access control strategy, query broker, network security, xml, in-broker access control deployment, control deployment, towards efficient end-to-end performance, information brokerage system, taken-for-granted access control deployment, data source side access control, distributed xml database system, query brokering, access control deployment strategy, data source side access, in-broker access control, telecommunication security, information brokerage systems, xml brokerage system, distributed databases, query processing, security of data, access control, system performance, xml database, data security, information security, database systems, xml document]",In-broker access control: towards efficient end-to-end performance of information brokerage systems
7,"[information technology, simulation, transportation, objective function, dynamic programming, state space, sensor network, elevators, uncertainty, testing, normal operator, look ahead, lagrangian relaxation, local search]",Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation
8,"[sinhala language, standarisation, complex script computing]",Sinhala Computing in Early Stage - Sri Lanka Experience.
9,"[application specific integrated circuits, integrated circuit layout, logic design, logic gates, network routing, 2-input NAND gates, ASIC, DFM, cell regular matrix, circuit routability, connection predictability, regular layout approach]",A Regular Layout Approach for ASICs


## Venue

In [0]:
# https://pypi.org/project/habanero/
from habanero import Crossref
cr = Crossref()

def getVenue(doi, venue):
    result = [venue['name_d'], None]
    try:
        query = cr.works(ids = doi)['message']['event']
        result[0] = query['name']
        result[1] = query['location']
    except:
        pass
    return result

getVenueUDF = udf(getVenue, ArrayType(StringType()))

In [0]:
venues_df = (filtered_df
            .limit(100)
            .select('title', "doi", 'venue')
            .withColumn("VENUE", getVenueUDF(F.col("doi"), F.col('venue')))
            .select('title',
                    F.col("VENUE")[0].alias("venue"),
                    F.col("VENUE")[1].alias("location")
                   )
             .dropna(subset="venue")
           )

venues = (venues_df.select("venue", "location")
            .dropDuplicates()
            .withColumn("venue_id", F.monotonically_increasing_id())
          )

venues = (venues_df.join(venues, ["venue", "location"])
           .select("venue_id", "venue", "location", "title")
           .dropDuplicates()
        )

display(venues)
venues_clean = venues.select("venue_id", "venue")
venues_clean.write.format("delta").mode("overwrite").saveAsTable("venues")

venue_id,venue,location,title
0,IGARSS 2008 - 2008 IEEE International Geoscience and Remote Sensing Symposium,"Boston, MA, USA",Validation of Coarse Resolution Microwave Soil Moisture Products
1,2007 IEEE Conference on Computer Vision and Pattern Recognition,"Minneapolis, MN, USA",Combining local and global motion models for feature point tracking
3,Mechanical Engineering and Information Technology (EMEIT),"Harbin, Heilongjiang, China",Design and realization of DICOM/HL7 gateway in PACS
4,"2008 IEEE International Conference on Systems, Man and Cybernetics (SMC)","Singapore, Singapore",Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.
6,ACE2006: International Conference in Advances in Computer Entertainment Technology,Hollywood California USA,To use PYA tool or not for learning Japanese hand alphabets
7,"IEEE International Conference on Sensor Networks, Ubiquitous, and Trustworthy Computing -Vol 1 (SUTC'06)","Taichung, Taiwan",In-broker access control: towards efficient end-to-end performance of information brokerage systems
8,2005 IEEE/RSJ International Conference on Intelligent Robots and Systems,"Edmonton, Alta., Canada",A simple low cost vision system for small unmanned VTOL vehicles.
9,2008 20th IEEE International Conference on Tools with Artificial Intelligence (ICTAI),"Dayton, OH",Improving Learner Performance with Data Sampling and Boosting
11,2013 Twenty-Eighth Annual IEEE/ACM Symposium on Logic in Computer Science (LICS 2013),"New Orleans, LA, USA",Dynamic Logic of Propositional Assignments: A Well-Behaved Variant of PDL
16,IUI-CADUI04: Intelligent User Interface,"Funchal, Madeira Portugal",Choosing when to interact with learners


## FieldOfStudy

In [0]:
def getFos(doi, fos):
    result = [] if not fos else fos
    query = []
    try:
        query = cr.works(ids = doi)['message']['subject']
    except:
        pass
    return [*result, *query]

getFosUDF = udf(getFos, ArrayType(StringType()))

In [0]:
fos_df = (filtered_df
          .limit(100)
          .select('title', "doi", 'fos')
          .withColumn("FOS", getFosUDF(F.col("doi"), F.col("fos")))
          .select('title', F.col('FOS').alias('fos'))
          .filter(F.size('fos') > 0)
           )

fos = (fos_df.select("fos")
            .dropDuplicates()
            .withColumn("fos_id", F.monotonically_increasing_id())
          )

fos = (fos_df.join(fos, ["fos"])
           .select("fos_id", "fos", "title")
           .dropDuplicates()
        )

display(fos)
fos_clean = fos.select("fos_id", "fos")
fos_clean.write.format("delta").mode("overwrite").saveAsTable("fos")

fos_id,fos,title
0,"List(Metadata, Computer aided instruction, Computer science, Rights Expression Language, Distance education, Learning object, Human–computer interaction, Artificial intelligence, Granularity, Intellectual property, Industrial property)",Research on DRM-enabled learning objects model
1,"List(Default logic, Model-based reasoning, Analytic reasoning, Artificial intelligence, Non-monotonic logic, Deductive reasoning, Reasoning system, Case-based reasoning, Mathematics, Qualitative reasoning, Artificial Intelligence, Linguistics and Language, Language and Linguistics)",Defaults and relevance in model-based reasoning
2,"List(Rapid prototyping, Interaction design, Computer science, Engineering management, Usability, Product design, Management science)",From behaviour to innovation at Nortel Networks
3,"List(Mobile computing, Smart environment, Telecommunications, Computer science, Computer security, Universal Plug and Play, Sun SPOT, Mobile phone, Web service, Wireless sensor network, The Internet)",Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.
4,"List(Tree-adjoining grammar, Context-sensitive grammar, Rule-based machine translation, Programming language, Permutation, Multimodal logic, Mathematical proof, Artificial intelligence, Categorial grammar, Natural language processing, Mathematics, Semantics, Linguistics and Language, Philosophy, Computer Science (miscellaneous))","Residuation, Structural Rules and Context Freeness"
5,"List(Political science, General Computer Science)",A retrospective look at PD projects
6,"List(Computer access control, World Wide Web, XML, Computer science, Role-based access control, Information security, XML database, Discretionary access control, Access control, Physical access)",In-broker access control: towards efficient end-to-end performance of information brokerage systems
7,"List(Motion control, Workspace, Computer science, Mechanical engineering, Design tool, Soft robotics, Robot, Human–robot interaction, Mobile robot, Structural engineering, Actuator)",Design of soft robotic actuators using fluid-filled fiber-reinforced elastomeric enclosures in parallel combinations.
8,"List(Digital signal processing, Computer science, Parallel computing, Computational science, Emulation, Quantization (signal processing), Order of magnitude, Word length, Signal-flow graph)",Global Approach for Compiled Bit-True Simulation of DSP Systems
9,"List(Data mining, Discretization, Pattern recognition, Variable and attribute, Uncertain data, Rough set, Fuzzy set, Approximate reasoning, Artificial intelligence, Mathematics, Computational Theory and Mathematics, Information Systems, Algebra and Number Theory, Theoretical Computer Science)",Rough-Set Reasoning About Uncertain Data


## Date

In [0]:
def getDate(doi, year):
    formated = [None, None, None]
    try:
        query = cr.works(ids = doi)['message']['issued']['date-parts'][0]
        for i in [0, 1, 2]:
            formated[i] = query[i]
    except:
        pass
    if formated[0] == None:
        formated[0] = year 
    return formated

getDateUDF = udf(getDate, ArrayType(StringType()))

In [0]:
dates_df = (filtered_df
            .limit(100)
            .select('title', "year", "doi")
            .withColumn("Date", getDateUDF(F.col("doi"), F.col("year")))
            .select('title',
                    F.col("Date")[2].alias("day"),
                    F.col("Date")[1].alias("month"),
                    F.col("Date")[0].alias("year")
                   )
           )

dates = (dates_df.select([F.col("day").alias("day1"), F.col("month").alias("month1"), F.col("year").alias("year1")])
            .dropDuplicates()
            .withColumn("date_id", F.monotonically_increasing_id())
          )

dates = (dates_df.join(dates, ((dates_df.day.eqNullSafe(dates.day1)) & 
                               (dates_df.month.eqNullSafe(dates.month1)) &
                               (dates_df.year.eqNullSafe(dates.year1))))
           .select("date_id", "day", "month", "year", "title")
           .dropDuplicates()
        )

display(dates)

dates_clean = dates.select("date_id", "day", "month", "year")
dates_clean.write.format("delta").mode("overwrite").saveAsTable("dates")

date_id,day,month,year,title
0,10,11,2007,Consistent Sets of Secondary Structures in Proteins
1,null,12,2002,A new feature extraction technique for on-line recognition of handwritten alphanumeric characters
2,1,6,2004,King John Divided
3,1,10,2010,Peer Assist Live Streaming Overlay for Next-Generation-Networks
4,null,null,2004,Paramecium: Assembling Raw Nodes into Composite Cells
5,null,null,2009,Radar Signal Retrodiffusion by Water Surface
6,null,10,2008,Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.
7,null,null,2005,Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation
8,null,10,2013,Weighted census transform for feature representation
9,null,10,1996,A signaling and control architecture for mobility support in wireless ATM networks


## Language

In [0]:
display(raw_df.groupBy("lang").count())
display(filtered_df.groupBy("lang").count())

lang,count
en,249984
zh,16


lang,count
en,244930


In [0]:
lang_df = filtered_df.select("title", "lang")

lang = (lang_df.select("lang")
            .dropDuplicates()
            .withColumn("lang_id", F.monotonically_increasing_id())
          )

lang = (lang_df.join(lang, ["lang"])
           .select("lang_id", "lang", "title")
           .dropDuplicates()
        )

display(lang)

lang_clean = lang.select("lang_id", "lang")
lang_clean.write.format("delta").mode("overwrite").saveAsTable("lang")

lang_id,lang,title
0,en,An adaptive technique for content-based image retrieval
0,en,Controlling Self-Organising Software Applications with Archetypes
0,en,Non-Automatizability of Bounded-Depth Frege Proofs
0,en,"Buffer Overflows und Format-String-Schwachstellen - Funktionsweisen, Exploits und Gegenmaßnahmen"
0,en,Empowering iso-surfaces with volume data
0,en,Partitioning and Retiming of Multi-Dimensional Systems
0,en,"A Comparison Between Contour Elevation Data Sources for DEM Creation and Soil Carbon Prediction, Coshocton, Ohio"
0,en,A pairing-based publicly verifiable secret sharing scheme.
0,en,A relational model for unstructured documents
0,en,A survey on applications of the harmony search algorithm.


## MAIN TABLE

In [0]:
main_table = (publications.join(authors, ["title"])
              .join(organization, ["author"], "left")
              .join(types, ["title"])
              .join(keywords, ["title"])
              .join(venues, ["title"])
              .join(fos, ["title"])
              .join(dates, ["title"])
              .join(lang, ["title"])
              .select("publ_id", "author_id", "org_id", "type_id", "keyw_id", "venue_id", "fos_id", "date_id", "lang_id", F.col("rank").alias("author_rank"))
              )
display(main_table)
main_table.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("DBLP_fact_table")

publ_id,author_id,org_id,type_id,keyw_id,venue_id,fos_id,date_id,lang_id,author_rank
8,0,null,2,66,11,63,25,0,2
59,19,null,2,10,0,49,47,0,6
59,29,null,2,10,0,49,47,0,4
4,42,60,2,55,48,74,7,0,7
21,43,null,2,33,18,90,23,0,1
4,48,60,2,55,48,74,7,0,3
2,62,null,2,69,8,27,7,0,1
59,78,null,2,10,0,49,47,0,3
3,79,70,2,64,33,84,48,0,3
39,81,36,2,17,16,75,34,0,1
